In [1]:
from __future__ import annotations

import jax
import jax.numpy as jnp

import hodel.dismech as dismech

jax.config.update("jax_enable_x64", True)

geom = dismech.Geometry(
    rod_r0=0.001,
    shell_h=0,
)

mat = dismech.Material(
    density=1200,
    youngs_rod=2e6,
    youngs_shell=0,
    poisson_rod=0.5,
    poisson_shell=0,
)

"""# TODO: Figure out how sim params works for HoDEL
sim_params = dismech.SimParams(
    max_iter=30,
    tol=1e-4,
    ftol=1e-4,
    dtol=1e-2,
)"""

mesh = dismech.Mesh.from_txt("rod.txt")
top, state, mass, triplets = dismech.from_legacy(mesh, geom, mat)

In [2]:
from dataclasses import dataclass
from jax.tree_util import register_dataclass

import jaxtyping

import hodel


def get_indices(
    q: jax.Array,
    top: dismech.Connectivity,
    fixed_nodes: jax.Array | None = None,
    fixed_edges: jax.Array | None = None,
) -> tuple[jax.Array, jax.Array]:
    nodes = (
        jnp.array([]) if fixed_nodes is None else dismech.map_node_to_dof(fixed_nodes)
    )
    edges = jnp.array([]) if fixed_edges is None else top.edge_dofs[0] + fixed_edges
    idx_b = jnp.union1d(nodes, edges)
    return idx_b, jnp.setdiff1d(jnp.arange(q.shape[0]), idx_b)


idx_b, idx_f = get_indices(
    state.q, top, jnp.array([0, 1, 2, 18, 19, 20]), jnp.array([0, 1, 18, 19])
)


@register_dataclass
@dataclass(frozen=True)
class TripletAux:
    """parametrized external force."""

    top: dismech.Connectivity  # for state.update()
    idx_f: jax.Array
    idx_b: jax.Array


def get_gravity(mass: jax.Array, g: jax.Array = jnp.array([0.0, 0.0, -9.81])):
    return mass * jnp.concat(
        [
            jnp.tile(g, mesh.nodes.shape[0]),
            jnp.zeros(mesh.edges.shape[0]),
        ]
    )


def get_W(lambda_: jax.Array, aux: TripletAux):
    """Just gravity."""
    return get_gravity(mass)[aux.idx_f]


def fixed_0(lambda_: jax.Array, aux: TripletAux) -> jax.Array:
    """Compress and contort."""
    return state.q[aux.idx_b] + lambda_ * jnp.concat(
        [
            jnp.array([0.05, 0.0, 0, 0.05, 0.0, 0, 0.05, 0.0, 0]),
            jnp.zeros(aux.idx_b.shape[0] - 13),
            jnp.array([1.0, 1.0, 0.0, 0.0]),
        ]
    )


def get_q(xf: jax.Array, xb: jax.Array, aux: TripletAux) -> jax.Array:
    """Helper to construct q from xf and xb."""
    q = jnp.empty((aux.idx_f.shape[0] + aux.idx_b.shape[0]), xf.dtype)
    return q.at[aux.idx_f].set(xf).at[aux.idx_b].set(xb)


def update_state(
    xf: jax.Array, xb: jax.Array, aux: TripletAux, carry: dismech.StaticState
) -> dismech.StaticState:
    """Construct q from xf and xb and update state."""
    q = get_q(xf, xb, aux)
    carry_new = carry.update(q, aux.top)
    return carry_new


def get_batch_energy(
    xf: jax.Array,
    xb: jax.Array,
    Theta: jaxtyping.PyTree,
    aux: TripletAux,
    carry: dismech.StaticState,
) -> jax.Array:
    q = get_q(xf, xb, aux)
    state = carry.update(q, aux.top)
    return jnp.sum(jax.vmap(lambda t: t.get_energy(state, Theta))(triplets))


xf0 = state.q[idx_f]
aux = TripletAux(top, idx_f, idx_b)
sim = hodel.HODEL(
    get_batch_energy, get_W_fn=get_W, get_xb_fn=fixed_0, carry_fn=update_state
)

In [5]:
lambdas = jnp.linspace(0, 1.0, 100)
xf_star = sim.sim(lambdas, xf0, None, aux, state, nsteps=10)
xb_star = jax.vmap(fixed_0, (0, None))(lambdas, aux)
qs = jax.vmap(get_q, (0, 0, None))(xf_star, xb_star, aux)
dismech.animate(lambdas, qs, top)

err: 6.046404903843583e-14
err: 6.42856900400105e-14
err: 7.290036596463542e-14
err: 7.657186934045415e-14
err: 6.18506844489232e-14
err: 7.618114365659955e-14
err: 6.791226936226058e-14
err: 5.820532324689539e-14
err: 4.0843093171865633e-14
err: 5.939330721715859e-14
err: 6.995810023165573e-14
err: 5.658898897704044e-14
err: 7.359868750373854e-14
err: 5.1691521006259405e-14
err: 5.230229364941132e-14
err: 5.36756529816764e-14
err: 6.167608954670735e-14
err: 7.57402548358479e-14
err: 6.734189132348388e-14
err: 5.61118634943354e-14
err: 4.203414040744014e-14
err: 3.8259214721575616e-14
err: 5.5275721779197845e-14
err: 4.906666852514934e-14
err: 5.037063029148972e-14
err: 4.998032299186768e-14
err: 4.501425579520668e-14
err: 5.07809972185769e-14
err: 6.115970609325855e-14
err: 5.5463244860086987e-14
err: 5.597810569890439e-14
err: 4.585516137675247e-14
err: 5.331312260324422e-14
err: 7.674708033008255e-14
err: 7.792672149043834e-14
err: 5.706863735619504e-14
err: 4.8108281798320234e-14
e